# Imports

In [ ]:
import json
import random
from pathlib import Path

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import Markdown
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from tqdm import tqdm

https://huggingface.co/spaces/mteb/leaderboard

Try 2 embeddings models:
1. USER-bge-m3 (360M): https://huggingface.co/deepvk/USER-bge-m3
2. ru-en-RoSBERTA (404M): https://huggingface.co/ai-forever/ru-en-RoSBERTa

# Dealing with legal practices docs tags

In [29]:
with open("/home/deniskirbaba/Documents/legal-ai/SUDACT_DATA_FULL.json") as meta_file:
    meta = json.loads(meta_file.read())

In [30]:
len(meta), meta[0].keys()

(130, dict_keys(['url', 'tags', 'documents']))

In [11]:
meta[0]

{'url': 'https://sudact.ru/practice/po-zashite-prav-potrebitelej/',
 'tags': ['Административный кодекс',
  'По защите прав потребителей',
  'Судебная практика по защите прав потребителей\nРешения судов, основанные на применении норм закона "О защите прав потребителей".\r\n\r\nЗакон "О защите прав потребителей"  перейти\xa0'],
 'documents': ['KvUCynva70VO',
  'sktFZwX2vJ6t',
  '2SKGOeAhZoJ',
  '3OmuKC3utXYw',
  '3oEVJiIQT2NB',
  'jw4v6ipvCpJj',
  'sCB5zpzly6on',
  'TJQGohE4D1rV',
  'Sc1T3fJioVVZ',
  'zOvzWK79z0QX',
  'O0lpfvACLLLW',
  'x081otwbcCUl',
  'TAFmwox2RExa',
  'nXjBk7v3aEW1',
  'JpOFIeDHJE6n',
  'xGBCgFBxluHo',
  'q6lJnjDXH5WE',
  'AmBpsmVrF9Yw',
  '9EzVhSZ4eFU6',
  'NBbl0Glap5ov',
  '3upYOEUsOC6X',
  'NgQBEXsutcV',
  'RKP8YC1IrAod',
  'Gyvo6qzyGbG',
  'V5xlIeP3cglC',
  'UIKKYQ6DydQl',
  'wNd1oHkB9dZR',
  '57KHl6X5ZxUv',
  'FkGrGRvovNWP',
  'SPKWCgZmao55',
  'wBVcgUFC0w18',
  'c5SmGZhKwQbC',
  'ZqZn1DH8dqxP',
  'hapJQyoA5VgP',
  'B5Kah7ur3Cdr',
  'uzxKzKzAv7oX',
  'Elg6HyByXwN

!!! note that one docUID can be correspond to multiple tags

In [13]:
tags = [cat["tags"] for cat in meta]

In [16]:
np.array([len(t) for t in tags])

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [22]:
np.unique([t[0] for t in tags])

array(['Административный кодекс', 'Гражданский кодекс',
       'Уголовный кодекс'], dtype='<U23')

In [23]:
np.unique([t[1] for t in tags])

array(['Амнистия', 'Доказательства',
       'Злоупотребление должностными полномочиями',
       'Иностранные граждане', 'Клевета', 'Коммерческий подкуп',
       'Контрабанда', 'Меры пресечения', 'Нарушение прав инвалидов',
       'Нарушение правил дорожного движения',
       'Незаконное получение кредита', 'Незаконное предпринимательство',
       'Оскорбление',
       'Осуществление предпринимательской деятельности без регистрации или без разрешения',
       'Персональные данные', 'По ДТП (невыполнение требований при ДТП)',
       'По ДТП (причинение легкого или средней тяжести вреда здоровью)',
       'По вымогательству', 'По грабежам', 'По делам о хулиганстве',
       'По делам об изнасиловании', 'По делам об убийстве',
       'По защите прав потребителей',
       'По коррупционным преступлениям, по взяточничеству', 'По кражам',
       'По лишению прав за "пьянку" (управление ТС в состоянии опьянения, отказ от освидетельствования)',
       'По лишению прав за обгон, "встречку"', 'По 

In [24]:
np.unique([t[2] for t in tags])

array(['Алименты в твердой денежной сумме',
       'Амнистия - судебная практика\nРешения судов, основанные на применении нормы статьи 84 Уголовного кодекса Российской Федерации.\r\nСт. 84 УК РФ. Амнистия  перейти\xa0',
       'Взыскание убытков', 'Возмещение убытков',
       'Восстановление срока принятия наследства ',
       'Выселение из квартиры', 'Выслуга лет',
       'Гражданско-правовой договор ', 'Добросовестный приобретатель',
       'Договор ренты',
       'Доказательства - судебная практика\nРешения судов, основанные на применении нормы статьи 74 Уголовно-процессуального кодекса Российской Федерации.\r\nСт. 74 УПК РФ. Доказательства  перейти\xa0',
       'Долг по расписке, по договору займа', 'Задаток',
       'Защита деловой репутации юридического лица, защита чести и достоинства гражданина',
       'Злоупотребление должностными полномочиями – судебная практика\nРешения судов, основанные на применении нормы статьи 285 Уголовного кодекса Российской Федерации.\r\nСт. 285 УК Р

In [26]:
np.unique([t[3] for t in tags if len(t) > 3])

array(['Алименты в твердой денежной сумме - судебная практика\nРешения судов, основанные на применении нормы статьи 83 Семейного кодекса Российской Федерации.\r\nСт. 83 СК РФ. Взыскание алиментов на несовершеннолетних детей в твердой денежной сумме \r\nперейти\xa0',
       'Взыскание убытков - судебная практика\nРешения судов, основанные на применении нормы статьи 393 Гражданского кодекса Российской Федерации.\r\nCт. 393 ГК РФ. Обязанность должника возместить убытки перейти\xa0',
       'Возмещение убытков - судебная практика\nРешения судов, основанные на применении нормы статьи 15 Гражданского кодекса Российской Федерации.\r\nСт. 15 ГК РФ. Возмещение убытков  перейти\xa0',
       'Восстановление срока принятия наследства - судебная практика\nРешения судов, основанные на применении нормы статьи 1155 Гражданского кодекса Российской Федерации.\r\nСт. 1155 ГК РФ. Принятие наследства по истечении установленного срока перейти\xa0',
       'Выселение из квартиры - судебная практика\nРешения 

In [36]:
# drop the ` перейти\xa0` from everywhere
for m in tqdm(meta):
    m["tags"][-1] = m["tags"][-1].replace(" перейти\xa0", "")

100%|██████████| 130/130 [00:00<00:00, 244291.90it/s]


We have 3 top groups: аднимистративный, гражданский, уголовный кодексы.  
And the other info is just theme.  

So let's create table with mappings from docUID: кодекс, тематика.

In [ ]:
codex_to_idx = {i: codex for i, codex in enumerate(meta)}

In [42]:
codex_to_idx = {}
theme_to_idx = {}
docs_meta = []

for m in tqdm(meta):
    codex = m["tags"][0]
    if codex not in codex_to_idx.keys():
        codex_to_idx[codex] = len(codex_to_idx)

    theme = "\n".join(m["tags"][1:])
    if theme not in theme_to_idx.keys():
        theme_to_idx[theme] = len(theme_to_idx)

    for doc_uid in m["documents"]:
        docs_meta.append(
            {"docUID": doc_uid, "codex": codex_to_idx[codex], "theme": theme_to_idx[theme]}
        )

100%|██████████| 130/130 [00:00<00:00, 2249.16it/s]


In [119]:
idx_to_codex = {i: codex for codex, i in codex_to_idx.items()}
idx_to_theme = {i: theme for theme, i in theme_to_idx.items()}

In [46]:
pd_docs_meta = pd.DataFrame(data=docs_meta)
pd_docs_meta

,docUID,codex,theme
0,KvUCynva70VO,0,0
1,sktFZwX2vJ6t,0,0
2,2SKGOeAhZoJ,0,0
3,3OmuKC3utXYw,0,0
4,3oEVJiIQT2NB,0,0
...,...,...,...
62989,CLt5hOtEJkUU,2,129
62990,tN5v3mtS7Idd,2,129
62991,Cxh590D85V1A,2,129
62992,ztBruCSUZPmc,2,129


In [47]:
codex_to_idx

{'Административный кодекс': 0, 'Уголовный кодекс': 1, 'Гражданский кодекс': 2}

In [48]:
theme_to_idx

{'По защите прав потребителей\nСудебная практика по защите прав потребителей\nРешения судов, основанные на применении норм закона "О защите прав потребителей".\r\n\r\nЗакон "О защите прав потребителей" ': 0,
 'Оскорбление\nОскорбление - судебная практика\nРешения судов, основанные на применении нормы статьи 5.61 Кодекса Российской Федерации об административных правонарушениях.\r\nСт. 5.61 КОАП РФ. Оскорбление': 1,
 'Нарушение прав инвалидов\nНарушение прав инвалидов - судебная практика\nРешения судов, основанные на применении нормы статьи 5.42 Кодекса Российской Федерации об административных правонарушениях.\r\nСт. 5.42 КОАП РФ. Нарушение прав инвалидов в области трудоустройства и занятости': 2,
 'Незаконное получение кредита\nНезаконное получение кредита - судебная практика\nРешения судов, основанные на применении нормы статьи 14.11 Кодекса Российской Федерации об административных правонарушениях.\r\nСт. 14.11 КОАП РФ. Незаконное получение кредита или займа ': 3,
 'Осуществление предп

# Create random subset of all docs 

In [72]:
docs_full_path = Path("/home/deniskirbaba/Documents/legal-ai/OUTPUT_PARSE")

docs_full_list = list(docs_full_path.iterdir())
n_full = len(docs_full_list)
print(f"Number of docs: {n_full}")

Number of docs: 51341


In [73]:
subset_ratio = 0.01
n_subset = max(1, int(n_full * subset_ratio))
print(f"Number of docs in subset: {n_subset}")

Number of docs in subset: 513


In [74]:
random.seed(42)
subset_files = random.sample(docs_full_list, n_subset)

# Load docs, process, split

In [ ]:
def parse_html(html: str) -> str:
    """
    Parse HTML doc to raw text. Also extract only data from <body> tags.

    Adds a new line for 'br',  'p', 'h1', 'h2', 'h3', 'h4','tr', 'th' tags
    and also a new line in front of text for li elements (bullet lists).
    """
    soup = BeautifulSoup(html, features="html.parser").body

    text = ""
    for e in soup.descendants:
        if isinstance(e, str):
            text += e.strip()
        elif e.name in ["br", "p", "h1", "h2", "h3", "h4", "tr", "th"]:
            text += "\n"
        elif e.name == "li":
            text += "\n- "
    return text

In [ ]:
processed_docs = []
for doc_path in tqdm(subset_files):
    with open(doc_path, "r") as f:
        html_doc = f.read()
    processed_docs.append(parse_html(html_doc))

100%|██████████| 513/513 [00:05<00:00, 91.77it/s] 


In [77]:
len(processed_docs)

513

In [81]:
Markdown(processed_docs[124][:2500])


Решение № 2-239/2023 2-239/2023~М-208/2023 М-208/2023 от 28 июля 2023 г. по делу № 2-239/2023Кайбицкий районный суд (Республика Татарстан ) - Гражданское
/
УИД 16RS0015-01-2023-000255-94

Копия Дело №2-239/2023


РЕШЕНИЕ

Именем Российской Федерации

28 июля 2023 года с. Большие Кайбицы

Кайбицкий районный суд Республики Татарстан

в составе:

председательствующего судьи Нигматзяновой Э.А.,

при секретаре судебного заседания Хузиной Г.И.,

рассмотрев в открытом судебном заседании в зале суда гражданское дело по исковому заявлению Исполнительного комитета Маломеминского сельского поселения Кайбицкого муниципального района Республики Татарстан к Исполнительному комитету Кайбицкого муниципального района Республики Татарстан о признании имущества выморочным, признании права собственности на выморочное имущество,

Установил:

Исполнительный комитет Маломеминского сельского поселения Кайбицкого муниципального района Республики Татарстан обратилось в суд с исковым заявлением к исполнительному комитету Кайбицкого муниципального района Республики Татарстан о признании имущества выморочным, признании права собственности на выморочное имущество, мотивируя тем, что ДД.ММ.ГГГГ умерла К.М.В., которой на праве собственности принадлежало недвижимое имущество в виде земельного участка, площадью 2400 кв.м с кадастровым номером №, расположенного по адресу: <адрес>. После смерти К.М.В. никто в права наследования не вступил, соответственно имущество является выморочным, право собственности на неё может быть признано за истцом.

Просят признать имущество, принадлежавшее К.М.В. выморочным; признать право собственности на них за муниципальным образованием «Маломеминское сельское поселение Кайбицкого муниципального района Республики Татарстан».

Истец – руководитель исполнительного комитета Маломеминского сельского поселения Кайбицкого муниципального района Республики Татарстан ФИО1 в судебном заседании исковые требования поддержала и дала объяснения соответствующие исковому заявлению.

Ответчик – представитель исполнительного комитета Кайбицкого муниципального района Республики Татарстан в судебное заседание не явился, извещен судом надлежащим образом.

Третье лицо, не заявляющее самостоятельные требования относительно предмета спора – нотариус Кайбицкого нотариального округа Республики Татарстан ФИО2 в судебное заседание не явился, извещен судом надлежащим образом.

Выслушав доводы представителя истца, исследовав материалы дела, суд приходит к следующему.

В соответствии со ст.

`USER-bge-m3` model has maximum input length of tokens = 8192, it's tokenizer has vocab size = 46166, so i guess we can set chunk size of splitter = 10000 symbols.

also we can try using tokensplitter from langchain: https://python.langchain.com/docs/how_to/split_by_token/

In [152]:
CHUNK_SIZE = 10000
CHUNK_OVERLAP_FACTOR = 0.1

In [153]:
rec_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n\n", "\n\n", "\n", " ", ""],
    chunk_size=CHUNK_SIZE,  #  need to pick value based on context of our model
    chunk_overlap=int(CHUNK_SIZE * CHUNK_OVERLAP_FACTOR),
    length_function=len,
    is_separator_regex=False,
)

In [ ]:
# metadata for our subset of docs
subset_doc_uids = [path.stem for path in subset_files]
subset_meta = []

for doc_iud in tqdm(subset_doc_uids, leave=False):
    meta_series = pd_docs_meta.loc[pd_docs_meta["docUID"] == doc_iud, ["codex", "theme"]]
    subset_meta.append(
        {
            "uid": doc_iud,
            "codex": [idx_to_codex[i] for i in meta_series["codex"]],
            "theme": [idx_to_theme[i] for i in meta_series["theme"]],
        }
    )
subset_meta[:5]

[{'uid': '8AjkyAm8BeN5',
  'codex': ['Гражданский кодекс'],
  'theme': ['Судебная практика по жилищным спорам и ЖКХ\nПо ТСЖ\nСудебная практика по ТСЖ\nРешения судов, основанные на применении норм статей 135, 136, 137, 138 Жилищного кодекса Российской Федерации.\r\n\r\nСт. 135 ЖК РФ. Товарищество собственников жилья \r\nСт. 136 ЖК РФ. Создание и государственная регистрация товарищества собственников жилья \r\nСт. 137 ЖК РФ. Права товарищества собственников жилья \r\nСт. 138 ЖК РФ. Обязанности товарищества собственников жилья ']},
 {'uid': '2BGLm5MicSl1',
  'codex': ['Гражданский кодекс'],
  'theme': ['Судебная практика по трудовым спорам\nМатериальная ответственность\nМатериальная ответственность - судебная практика\nРешения судов, основанные на применении нормы статьи 242 Трудового кодекса Российской Федерации.\r\nСт. 242 ТК РФ. Полная материальная ответственность работника \r\nперейти\xa0']},
 {'uid': 'UbXk7N0P4WmW',
  'codex': ['Уголовный кодекс'],
  'theme': ['Коммерческий подкуп\nК

In [155]:
chunks = rec_splitter.create_documents(processed_docs, subset_meta)
f"{len(chunks)} chunks was construct from {len(processed_docs)} HTML docs"

'1843 chunks was construct from 513 HTML docs'

In [ ]:
# let's observe the 1 HTML splits
for chunk in chunks:
    if chunk.metadata["uid"] == subset_doc_uids[0]:
        print("=" * 250)
        print(chunk.page_content)

Решение № 2-4476/2023 2-4476/2023~М-4066/2023 М-4066/2023 от 4 декабря 2023 г. по делу № 2-4476/2023Дзержинский районный суд г. Перми (Пермский край) - Гражданское
/
Дело № 2-4476/2023

59RS0001-01-2023-005391-67

ЗАОЧНОЕ
РЕШЕНИЕ

Именем Российской Федерации

г. Пермь 04 декабря 2023 года

Дзержинский районный суд г. Перми в составе:

председательствующего судьи Суворовой К.А.,

при секретаре Бурсиной В.В.,

с участием представителя истца ФИО4, действующей на основании доверенности,

рассмотрев в открытом судебном заседании гражданское дело по иску ТСЖ «Союз» к ФИО1, ФИО2 о взыскании задолженности за ЖКУ, капитальный ремонт, пеней, судебных расходов,

УСТАНОВИЛ:

Истец обратился в суд с иском о взыскании с ответчиков задолженности по оплате жилищно-коммунальных услуг за период с Дата по Дата в размере по 30551,09 руб. с каждого, по оплате взносов за капитальный ремонт за период с Дата по Дата в размере по 9065,57 руб. с каждого, пеней за период с Дата по Дата за несвоевременную оплату 

# Set up embedding model

In [158]:
EMBEDDING_MODEL_NAME = "deepvk/USER-bge-m3"

In [159]:
emb_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    encode_kwargs={"normalize_embeddings": True},
)

/home/deniskirbaba/miniconda3/envs/practicing-rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

# ChromaDB

Each document in ChromaDB has to have unique id (we can construct it like - filename + #of chunk) and metadata dict - we need to store tags and the #of chunk aswell.